<a href="https://colab.research.google.com/github/JSJeong-me/KOSA-Transfer_Learning/blob/main/Pytorch_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning
- Pretrained ResNet50
- Custom dataset
- Add fully connected layer 

In [1]:
!unzip images.zip

Archive:  images.zip
   creating: images/dogs/
  inflating: images/dogs/3b1a8d3fca2adb68a7a908b73005764a.jpg  
  inflating: images/dogs/dog-650299.jpg  
  inflating: images/dogs/retriever.jpg  
   creating: images/onepun/
  inflating: images/onepun/1454.jpg  
  inflating: images/onepun/5320.jpg  
  inflating: images/onepun/5720.jpg  


## 1. Settings
### 1) Import required libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt

### 2) Hyperparameters

In [3]:
batch_size = 6
learning_rate = 0.001
epoch = 50
num_category = 2

## 2. Data
### 1) Load images from folder

In [4]:
# Input pipeline from a folder containing multiple folders of images
# we can check the classes, class_to_idx, and filename with idx

img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            ]))

print(img_data.classes)
print(img_data.class_to_idx)
print(img_data.imgs)

['dogs', 'onepun']
{'dogs': 0, 'onepun': 1}
[('./images/dogs/3b1a8d3fca2adb68a7a908b73005764a.jpg', 0), ('./images/dogs/dog-650299.jpg', 0), ('./images/dogs/retriever.jpg', 0), ('./images/onepun/1454.jpg', 1), ('./images/onepun/5320.jpg', 1), ('./images/onepun/5720.jpg', 1)]


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:935: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +


### 2) Set data loader

In [5]:
# After we get the list of images, we can turn the list into batches of images
# with torch.utils.data.DataLoader()

img_batch = DataLoader(img_data, batch_size=batch_size,
                            shuffle=True, num_workers=2)

for img,label in img_batch:
    print(img.size())
    print(label)

torch.Size([6, 3, 224, 224])
tensor([1, 1, 0, 1, 0, 0])


## 3. Model & Optimizer
### 1) ResNet

In [6]:
# https://discuss.pytorch.org/t/module-children-vs-module-modules/4551
# children() -> immediate children modules 
# modules() -> iterate all modules

resnet = models.resnet50(pretrained=True)

for name,module in resnet.named_children():
    print(name)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


### 2) Fully Connected Model

In [7]:
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet,self).__init__()
        self.layer0 = nn.Sequential(*list(resnet.children())[0:-1])
        self.layer1 = nn.Sequential(
            nn.Linear(2048,500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Linear(500,num_category),
            nn.ReLU()
        )
        
    def forward(self,x):
        out = self.layer0(x)
        out = out.view(batch_size,-1)
        out= self.layer1(out)
        
        return out

### 3) Model on GPU

In [8]:
model = Resnet().cuda()

for params in model.layer0.parameters():
    params.require_grad = False
    
for params in model.layer1.parameters():
    params.requires_grad = True

In [9]:
for name in model.children():
    print(name)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


### 4) Loss & Optimizer

In [10]:
# define loss func & optimizer
# model.parameters() also works because of the cell right above

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.layer1.parameters(),lr=learning_rate) 

## 4. Train

In [11]:
for i in range(epoch):
    for img,label in img_batch:
        img = Variable(img).cuda()
        label = Variable(label).cuda()

        optimizer.zero_grad()
        output = model(img)
        loss = loss_func(output,label)
        loss.backward()
        optimizer.step()

    if i % 10 ==0:
        print(loss)

tensor(0.6021, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3503, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3474, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3473, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3472, device='cuda:0', grad_fn=<NllLossBackward0>)


## 6. Test

In [12]:
model.eval()
correct = 0
total = 0

for img,label in img_batch:
    img = Variable(img).cuda()
    label = Variable(label).cuda()
    
    output = model(img)
    _, pred = torch.max(output.data,1)
    
    total += label.size(0)
    correct += (pred == label.data).sum()   

print("Accuracy: {}".format(correct/total))

Accuracy: 1.0
